# 02. Bedrock Client 테스트

AWS Bedrock Knowledge Base와 RetrieveAndGenerate API를 테스트합니다.

## 1. 환경 설정 및 라이브러리 import

In [33]:
import os
import sys
import boto3
import json
import time
from dotenv import load_dotenv
from botocore.exceptions import ClientError

# 환경 변수 로드
load_dotenv(override=True)

# AWS 클라이언트 초기화
region = os.getenv('AWS_REGION', 'us-west-2')
bedrock_runtime_client = boto3.client('bedrock-runtime', region_name=region)
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime', region_name=region)  # RetrieveAndGenerate용
bedrock_agent_client = boto3.client('bedrock-agent', region_name=region)  # Ingestion Job용
s3_client = boto3.client('s3', region_name=region)

print(f"✅ AWS 클라이언트 초기화 완료 (Region: {region})")

✅ AWS 클라이언트 초기화 완료 (Region: us-west-2)


## 2. 환경 변수 확인

In [36]:
# .env 파일에서 설정 로드
kb_id = os.getenv('BEDROCK_KNOWLEDGE_BASE_ID')
data_source_id = os.getenv('BEDROCK_DATA_SOURCE_ID')
s3_bucket_name = os.getenv('S3_BUCKET_NAME')
model_id = os.getenv('BEDROCK_MODEL_ID')
account_id = os.getenv('AWS_ACCOUNT_ID')
region = os.getenv('AWS_REGION')

print("🔍 환경 변수 확인:")
print(f"Knowledge Base ID: {kb_id}")
print(f"Data Source ID: {data_source_id}")
print(f"S3 Bucket: {s3_bucket_name}")
print(f"Model ID: {model_id}")
print(f"AWS_ACCOUNT_ID: {account_id}")
print(f"AWS_REGION: {region}")

if not all([kb_id, data_source_id, s3_bucket_name, model_id]):
    print("\n⚠️ 일부 환경 변수가 설정되지 않았습니다.")
    print("01_aws_setup.ipynb를 먼저 실행하세요.")
else:
    print("\n✅ 모든 환경 변수가 설정되었습니다.")

🔍 환경 변수 확인:
Knowledge Base ID: SVJFXS8QCG
Data Source ID: VMDVQHKIM1
S3 Bucket: my-rag-chatbot-5711d275
Model ID: anthropic.claude-3-7-sonnet-20250219-v1:0
AWS_ACCOUNT_ID: 191629784099
AWS_REGION: us-west-2

✅ 모든 환경 변수가 설정되었습니다.


## 3. Bedrock Client 클래스 정의

In [17]:
import logging

# 로깅 설정
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class BedrockClient:
    """AWS Bedrock Knowledge Base와 상호작용하는 클라이언트"""
    
    def __init__(self, knowledge_base_id, model_id, region='us-west-2'):
        self.knowledge_base_id = knowledge_base_id
        self.model_id = model_id
        self.region = region
        
        # Bedrock 클라이언트 초기화
        self.bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime', region_name=region)
        self.bedrock_runtime_client = boto3.client('bedrock-runtime', region_name=region)
        
        logger.info(f"BedrockClient 초기화 완료 - KB ID: {knowledge_base_id}")
    
    def retrieve_and_generate(self, query, session_id=None):
        """Knowledge Base에서 검색하고 응답 생성"""
        try:
            logger.info(f"🔍 Knowledge Base 검색 시작: {query[:50]}....")
            
            # RetrieveAndGenerate API 호출 파라미터
            params = {
                'input': {
                    'text': query
                },
                'retrieveAndGenerateConfiguration': {
                    'type': 'KNOWLEDGE_BASE',
                    'knowledgeBaseConfiguration': {
                        'knowledgeBaseId': self.knowledge_base_id,
                        'modelArn': f'arn:aws:bedrock:{self.region}::foundation-model/{self.model_id}'
                    }
                }
            }
            
            # 세션 ID가 있으면 추가
            if session_id:
                params['sessionId'] = session_id
            
            # API 호출
            response = self.bedrock_agent_runtime_client.retrieve_and_generate(**params)
            
            # 응답 파싱
            answer = response['output']['text']
            session_id = response.get('sessionId')
            citations = response.get('citations', [])
            
            # 소스 문서 정보 추출
            sources = []
            for citation in citations:
                for reference in citation.get('retrievedReferences', []):
                    location = reference.get('location', {})
                    s3_location = location.get('s3Location', {})
                    
                    source_info = {
                        'content': reference.get('content', {}).get('text', ''),
                        'filename': s3_location.get('uri', '').split('/')[-1] if s3_location.get('uri') else 'Unknown',
                        'uri': s3_location.get('uri', ''),
                        'confidence': reference.get('metadata', {}).get('confidence', 0)
                    }
                    sources.append(source_info)
            
            logger.info(f"✅ 검색 완료: {len(sources)}개 소스 발견")
            
            return {
                'answer': answer,
                'sources': sources,
                'session_id': session_id,
                'citations': citations
            }
            
        except ClientError as e:
            error_code = e.response['Error']['Code']
            error_message = e.response['Error']['Message']
            logger.error(f"❌ Bedrock API 오류 [{error_code}]: {error_message}")
            
            return {
                'answer': f"죄송합니다. 검색 중 오류가 발생했습니다: {error_message[:100]}...",
                'sources': [],
                'session_id': session_id,
                'error': True,
                'error_code': error_code
            }
            
        except Exception as e:
            logger.error(f"❌ 예상치 못한 오류: {str(e)}")
            
            return {
                'answer': f"죄송합니다. 시스템 오류가 발생했습니다: {str(e)[:100]}...",
                'sources': [],
                'session_id': session_id,
                'error': True,
                'error_code': 'SystemError'
            }

# BedrockClient 인스턴스 생성
if kb_id and model_id:
    bedrock_client = BedrockClient(kb_id, model_id, region)
    print("✅ BedrockClient 인스턴스 생성 완료")
else:
    print("⚠️ Knowledge Base ID 또는 Model ID가 없어 클라이언트를 생성할 수 없습니다.")
    bedrock_client = None

INFO:__main__:BedrockClient 초기화 완료 - KB ID: SVJFXS8QCG


✅ BedrockClient 인스턴스 생성 완료


## 4. 기본 연결 테스트

In [18]:
# Knowledge Base 연결 테스트
if bedrock_client:
    print("🔍 Knowledge Base 연결 테스트...")
    
    test_query = "안녕하세요. 연결 테스트입니다."
    result = bedrock_client.retrieve_and_generate(test_query)
    
    print(f"📝 응답: {result['answer'][:100]}...")
    print(f"📚 소스 개수: {len(result['sources'])}개")
    
    if result.get('error'):
        print(f"⚠️ 오류 발생: {result.get('error_code')}")
    else:
        print("✅ Knowledge Base 연결 성공")
else:
    print("⚠️ BedrockClient가 초기화되지 않아 테스트를 건너뜁니다.")

INFO:__main__:🔍 Knowledge Base 검색 시작: 안녕하세요. 연결 테스트입니다.....


🔍 Knowledge Base 연결 테스트...


INFO:__main__:✅ 검색 완료: 0개 소스 발견


📝 응답: Sorry, I am unable to assist you with this request....
📚 소스 개수: 0개
✅ Knowledge Base 연결 성공


## 5. 테스트 문서 업로드 및 Knowledge Base 동기화

In [21]:
import os
import time
from pathlib import Path

def upload_test_documents():
    """tests 디렉토리의 RAG 관련 문서들을 S3에 업로드"""
    
    print("📄 테스트 문서 업로드 시작...")
    
    # 업로드할 파일들 정의
    test_files = [
        {
            'local_path': 'tests/aws_rag_guide.md',
            's3_key': 'documents/aws_rag_guide.md',
            'content_type': 'text/markdown'
        }
    ]
    
    uploaded_files = []
    
    for file_info in test_files:
        local_path = file_info['local_path']
        s3_key = file_info['s3_key']
        content_type = file_info['content_type']
        
        # 파일 존재 확인
        if not os.path.exists(local_path):
            print(f"⚠️ 파일을 찾을 수 없습니다: {local_path}")
            continue
            
        try:
            # 파일 읽기
            with open(local_path, 'r', encoding='utf-8') as f:
                content = f.read()
            
            # S3에 업로드
            s3_client.put_object(
                Bucket=s3_bucket_name,
                Key=s3_key,
                Body=content.encode('utf-8'),
                ContentType=content_type
            )
            
            uploaded_files.append(os.path.basename(local_path))
            print(f"✅ {os.path.basename(local_path)} 업로드 완료")
            
        except Exception as e:
            print(f"❌ {local_path} 업로드 실패: {e}")
    
    print(f"\n📊 총 {len(uploaded_files)}개 테스트 문서 업로드 완료")
    return uploaded_files

# upload_test_documents 함수 정의 후
print(f"함수 정의 후: {bedrock_agent_client._service_model.service_name}")

def sync_knowledge_base(kb_id, data_source_id):
    """Knowledge Base와 S3 데이터 소스 동기화"""
    try:
        print("\n🔄 Knowledge Base 동기화 시작...")
        
        # Ingestion Job 시작
        response = bedrock_agent_client.start_ingestion_job(
            knowledgeBaseId=kb_id,
            dataSourceId=data_source_id,
            description='Test documents from tests directory'
        )
        
        job_id = response['ingestionJob']['ingestionJobId']
        print(f"📋 Ingestion Job ID: {job_id}")
        
        # Job 상태 모니터링
        print("⏳ 동기화 진행 중...")
        
        max_wait_time = 300  # 최대 5분 대기
        start_time = time.time()
        
        while time.time() - start_time < max_wait_time:
            job_status = bedrock_agent_client.get_ingestion_job(
                knowledgeBaseId=kb_id,
                dataSourceId=data_source_id,
                ingestionJobId=job_id
            )
            
            status = job_status['ingestionJob']['status']
            print(f"📊 상태: {status}")
            
            if status == 'COMPLETE':
                print("✅ Knowledge Base 동기화 완료!")
                return True
                
            elif status == 'FAILED':
                print("❌ Knowledge Base 동기화 실패")
                failure_reasons = job_status['ingestionJob'].get('failureReasons', [])
                for reason in failure_reasons:
                    print(f"   - {reason}")
                return False
                
            elif status in ['IN_PROGRESS', 'STARTING']:
                time.sleep(15)  # 15초 대기
            else:
                print(f"⚠️ 예상치 못한 상태: {status}")
                time.sleep(10)
        
        print("⏰ 동기화 대기 시간 초과")
        return False
                
    except Exception as e:
        print(f"❌ 동기화 오류: {e}")
        return False

# sync_knowledge_base 함수 정의 후  
print(f"sync 함수 정의 후: {bedrock_agent_client._service_model.service_name}")

# 디버깅: 클라이언트 타입 확인
print(f"bedrock_agent_client 타입: {type(bedrock_agent_client)}")
print(f"bedrock_agent_client 서비스: {bedrock_agent_client._service_model.service_name}")

# 실행 코드 시작 전
print(f"실행 코드 시작 전: {bedrock_agent_client._service_model.service_name}")

# 실행 코드
if not all([kb_id, data_source_id, s3_bucket_name]):
    print("❌ 필요한 환경 변수가 설정되지 않았습니다.")
    print("01_aws_setup.ipynb를 먼저 실행하세요.")
else:
    # 테스트 문서 업로드
    uploaded_files = upload_test_documents()
    
    if uploaded_files:
        # Knowledge Base 동기화
        sync_success = sync_knowledge_base(kb_id, data_source_id)
        
        if sync_success:
            print("\n🎉 테스트 문서가 Knowledge Base에 성공적으로 추가되었습니다!")
        else:
            print("\n⚠️ 동기화에 문제가 있었습니다.")
    else:
        print("\n❌ 업로드된 파일이 없어 동기화를 건너뜁니다.")



함수 정의 후: bedrock-agent
sync 함수 정의 후: bedrock-agent
bedrock_agent_client 타입: <class 'botocore.client.AgentsforBedrock'>
bedrock_agent_client 서비스: bedrock-agent
실행 코드 시작 전: bedrock-agent
📄 테스트 문서 업로드 시작...
✅ aws_rag_guide.md 업로드 완료

📊 총 1개 테스트 문서 업로드 완료

🔄 Knowledge Base 동기화 시작...
📋 Ingestion Job ID: 0IL1NPHXBE
⏳ 동기화 진행 중...
📊 상태: STARTING
📊 상태: COMPLETE
✅ Knowledge Base 동기화 완료!

🎉 테스트 문서가 Knowledge Base에 성공적으로 추가되었습니다!


## 6. Knowledge Base 검색 테스트

In [39]:
#  Knowledge Base 검색 테스트

def test_knowledge_base_search(query, kb_id, max_results=5):
    """Knowledge Base 검색 테스트"""
    try:
        print(f"🔍 검색 질문: {query}")
        print("="*50)
        
        response = bedrock_agent_runtime.retrieve(
            knowledgeBaseId=kb_id,
            retrievalQuery={'text': query},
            retrievalConfiguration={
                'vectorSearchConfiguration': {
                    'numberOfResults': max_results
                }
            }
        )
        
        print(f"📊 검색 결과: {len(response['retrievalResults'])}개 문서 발견")
        print()
        
        for i, result in enumerate(response['retrievalResults'], 1):
            print(f"📄 결과 {i}:")
            print(f"   점수: {result['score']:.4f}")
            print(f"   내용: {result['content']['text'][:200]}...")
            if 'location' in result:
                print(f"   출처: {result['location']['s3Location']['uri']}")
            print("-" * 40)
            
        return response['retrievalResults']
        
    except Exception as e:
        print(f"❌ 검색 오류: {e}")
        return None

# 테스트 실행
print("🚀 Knowledge Base 검색 테스트 시작")
print(f"📋 Knowledge Base ID: {kb_id}")
print(f"⏰ 테스트 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print()

# RAG에 대한 질문
test_queries = [
    "RAG가 무엇인가요? RAG의 정의와 특징을 설명해주세요.",
    "AWS Bedrock Knowledge Base는 어떻게 작동하나요?",
    "멀티모달 RAG 시스템의 장점은 무엇인가요?"
]

for i, query in enumerate(test_queries, 1):
    print(f"\n🔍 테스트 {i}:")
    results = test_knowledge_base_search(query, kb_id)
    
    if results:
        print(f"✅ 검색 성공! {len(results)}개 관련 문서 발견")
    else:
        print("❌ 검색 실패")
    
    print("\n" + "="*60 + "\n")

print("🎉 Knowledge Base 검색 테스트 완료!")


🚀 Knowledge Base 검색 테스트 시작
📋 Knowledge Base ID: SVJFXS8QCG
⏰ 테스트 시간: 2025-08-22 20:18:00


🔍 테스트 1:
🔍 검색 질문: RAG가 무엇인가요? RAG의 정의와 특징을 설명해주세요.
📊 검색 결과: 2개 문서 발견

📄 결과 1:
   점수: 0.5420
   내용: # 검색 증강 생성(RAG)이란 무엇인가요? [AWS 계정 생성](https://portal.aws.amazon.com/gp/aws/developer/registration/index.html?pg=what_is_header) 검색 증강 생성이란 무엇인가요? 검색 증강 생성이 중요한 이유는 무엇인가요? 검색 증강 생성의 이점은 무엇인가요? 검색 증강 생성은...
   출처: s3://my-rag-chatbot-5711d275/documents/aws_rag_guide.md
----------------------------------------
📄 결과 2:
   점수: 0.4663
   내용: RAG를 통해 LLM에 새 데이터를 더 비용 효율적으로 도입할 수 있습니다. 그리고 생성형 인공 지능(생성형 AI) 기술을 보다 폭넓게 접근하고 사용할 수도 있습니다. ### **최신 정보** LLM의 원본 훈련 데이터 소스가 요구 사항에 적합하더라도 연관성을 유지하기가 어렵습니다. 개발자는 RAG를 사용하여 생성형 모델에 최신 연구, 통계 또는 뉴스를 제...
   출처: s3://my-rag-chatbot-5711d275/documents/aws_rag_guide.md
----------------------------------------
✅ 검색 성공! 2개 관련 문서 발견



🔍 테스트 2:
🔍 검색 질문: AWS Bedrock Knowledge Base는 어떻게 작동하나요?
📊 검색 결과: 2개 문서 발견

📄 결과 1:
   점수: 0.5506
   내용: 확장된 프롬프트를 사용하면 대규모 언어 모델이 사용자 쿼리에 대한

## 7. RetrieveAndGenerate API 테스트

In [40]:
# 7번 섹션: RetrieveAndGenerate API 간단 테스트

def test_retrieve_and_generate(query, kb_id):
    """RetrieveAndGenerate API 테스트"""
    try:
        print(f"💬 질문: {query}")
        print("="*50)
        
        response = bedrock_agent_runtime.retrieve_and_generate(
            input={'text': query},
            retrieveAndGenerateConfiguration={
                'type': 'KNOWLEDGE_BASE',
                'knowledgeBaseConfiguration': {
                    'knowledgeBaseId': kb_id,
                    'modelArn': f'arn:aws:bedrock:{region}:{account_id}:inference-profile/us.{model_id}'
                }
            }
        )
        
        print(f"🤖 AI 답변:")
        print(response['output']['text'])
        print("\n" + "="*80 + "\n")
        
        return True
        
    except Exception as e:
        print(f"❌ 오류: {e}")
        return False

# 테스트 실행
print("🚀 RetrieveAndGenerate API 테스트")
print(f"📋 Knowledge Base ID: {kb_id}")
print()

test_queries = [
    "RAG가 무엇인가요?",
    "AWS Bedrock의 장점은 무엇인가요?"
]

for query in test_queries:
    success = test_retrieve_and_generate(query, kb_id)
    if success:
        print("✅ 성공!")
    else:
        print("❌ 실패!")

print("🎉 테스트 완료!")

🚀 RetrieveAndGenerate API 테스트
📋 Knowledge Base ID: SVJFXS8QCG

💬 질문: RAG가 무엇인가요?
🤖 AI 답변:
RAG(검색 증강 생성)는 대규모 언어 모델(LLM)의 출력을 최적화하여 응답을 생성하기 전에 훈련 데이터 소스 외부의 신뢰할 수 있는 기술 자료를 참조하도록 하는 프로세스입니다. 이는 LLM의 기능을 특정 도메인이나 조직의 내부 기술 자료로 확장하므로 모델을 다시 훈련할 필요가 없어 비용 효율적인 접근 방식입니다. RAG가 없는 경우 LLM은 사용자 입력을 바탕으로 훈련한 정보 또는 이미 알고 있는 정보를 기반으로 응답을 생성합니다. RAG는 사용자 입력을 활용하여 먼저 새 데이터 소스에서 정보를 가져오는 정보 검색 구성 요소를 도입합니다. 사용자 쿼리와 관련 정보가 모두 LLM에 제공되어 새로운 지식과 학습 데이터를 사용해 더 나은 응답을 생성할 수 있게 합니다.


✅ 성공!
💬 질문: AWS Bedrock의 장점은 무엇인가요?
🤖 AI 답변:
AWS Bedrock은 생성형 AI 애플리케이션을 구축할 수 있는 완전관리형 서비스로, 다음과 같은 장점을 제공합니다:

1. 개발 간소화: 고성능 기반 모델과 다양한 기능을 제공하여 생성형 AI 애플리케이션 개발을 간소화합니다.

2. 프라이버시 및 보안 유지: 애플리케이션 개발 시 프라이버시와 보안을 유지할 수 있습니다.

3. 지식 기반 활용: 클릭 몇 번으로 기반 모델(FM)을 검색 증강 생성(RAG)용 데이터 소스에 연결할 수 있습니다.

4. 자동화된 처리: 벡터 변환, 검색 및 개선된 출력 생성이 모두 자동으로 처리됩니다.


✅ 성공!
🎉 테스트 완료!


## 8. 채팅 기능 테스트 테스트

In [41]:
class RAGChatBot:
    """RAG 챗봇 클래스"""
    
    def __init__(self, kb_id, model_id, region, account_id):
        self.kb_id = kb_id
        self.model_id = model_id
        self.region = region
        self.account_id = account_id
        self.bedrock_agent_runtime = boto3.client('bedrock-agent-runtime', region_name=region)
        self.session_id = None
        self.chat_history = []
    
    def chat(self, message):
        """채팅 메시지 처리"""
        try:
            print(f"👤 사용자: {message}")
            
            config = {
                'input': {'text': message},
                'retrieveAndGenerateConfiguration': {
                    'type': 'KNOWLEDGE_BASE',
                    'knowledgeBaseConfiguration': {
                        'knowledgeBaseId': self.kb_id,
                        'modelArn': f'arn:aws:bedrock:{self.region}:{self.account_id}:inference-profile/us.{self.model_id}'
                    }
                }
            }
            
            # 세션 ID가 있으면 추가 (대화 연속성)
            if self.session_id:
                config['sessionId'] = self.session_id
            
            response = self.bedrock_agent_runtime.retrieve_and_generate(**config)
            
            answer = response['output']['text']
            self.session_id = response.get('sessionId')
            
            # 대화 히스토리에 추가
            self.chat_history.append({
                'user': message,
                'assistant': answer,
                'timestamp': datetime.now().strftime('%H:%M:%S')
            })
            
            print(f"🤖 AI: {answer}")
            
            # 참조 문서 표시 (간단히)
            if 'citations' in response and response['citations']:
                print(f"\n📚 참조 문서: {len(response['citations'])}개")
            
            print("-" * 60)
            return answer
            
        except Exception as e:
            print(f"❌ 채팅 오류: {e}")
            return None
    
    def show_history(self):
        """대화 히스토리 표시"""
        print("\n📜 대화 히스토리:")
        print("=" * 60)
        for i, chat in enumerate(self.chat_history, 1):
            print(f"[{chat['timestamp']}] 대화 {i}")
            print(f"👤 사용자: {chat['user']}")
            print(f"🤖 AI: {chat['assistant'][:100]}...")
            print("-" * 40)
    
    def clear_session(self):
        """세션 초기화"""
        self.session_id = None
        self.chat_history = []
        print("🔄 세션이 초기화되었습니다.")

# 챗봇 초기화
print("🚀 RAG 챗봇 초기화")
chatbot = RAGChatBot(kb_id, model_id, region, account_id)
print(f"📋 Knowledge Base ID: {kb_id}")
print(f"🤖 Model: {model_id}")
print(f"🔗 세션 준비 완료")
print("\n" + "=" * 60)

# 대화형 테스트
print("💬 대화형 RAG 테스트 시작")
print("(종료하려면 'quit' 입력)")
print("=" * 60)

# 미리 정의된 테스트 질문들
test_conversations = [
    "안녕하세요! RAG가 무엇인지 설명해주세요.",
    "그럼 기존 AI 모델과 어떤 차이가 있나요?",
    "AWS Bedrock에서 RAG를 구현하는 방법을 알려주세요.",
    "멀티모달 RAG의 장점은 무엇인가요?",
    "실제 프로젝트에 적용할 때 주의사항이 있나요?"
]

print("🎯 자동 테스트 대화:")
for i, question in enumerate(test_conversations, 1):
    print(f"\n🔄 테스트 {i}:")
    answer = chatbot.chat(question)
    if not answer:
        print("❌ 대화 실패")
        break
    
    # 잠시 대기 (선택사항)
    import time
    time.sleep(1)

# 대화 히스토리 표시
print("\n" + "=" * 60)
chatbot.show_history()

# 수동 대화 모드 (선택사항)
print("\n" + "=" * 60)
print("💬 수동 대화 모드 (Enter를 눌러 건너뛰기)")
print("질문을 입력하세요 (종료: 'quit'):")

try:
    while True:
        user_input = input("\n👤 사용자: ").strip()
        
        if not user_input:  # Enter만 누르면 종료
            break
            
        if user_input.lower() in ['quit', 'exit', '종료']:
            break
        
        answer = chatbot.chat(user_input)
        if not answer:
            print("❌ 오류가 발생했습니다. 다시 시도해주세요.")

except KeyboardInterrupt:
    print("\n\n🔄 대화가 중단되었습니다.")

print("\n🎉 채팅 기능 테스트 완료!")
print(f"📊 총 대화 수: {len(chatbot.chat_history)}개")
print("✅ RAG 챗봇이 정상적으로 작동합니다!")

🚀 RAG 챗봇 초기화
📋 Knowledge Base ID: SVJFXS8QCG
🤖 Model: anthropic.claude-3-7-sonnet-20250219-v1:0
🔗 세션 준비 완료

💬 대화형 RAG 테스트 시작
(종료하려면 'quit' 입력)
🎯 자동 테스트 대화:

🔄 테스트 1:
👤 사용자: 안녕하세요! RAG가 무엇인지 설명해주세요.
🤖 AI: RAG(검색 증강 생성)는 대규모 언어 모델(LLM)의 출력을 최적화하여 응답을 생성하기 전에 훈련 데이터 소스 외부의 신뢰할 수 있는 기술 자료를 참조하도록 하는 프로세스입니다. 이는 LLM의 기능을 특정 도메인이나 조직의 내부 기술 자료로 확장하므로 모델을 다시 훈련할 필요가 없는 비용 효율적인 접근 방식입니다. RAG가 없는 경우, LLM은 사용자 입력을 바탕으로 훈련한 정보나 이미 알고 있는 정보만을 기반으로 응답을 생성합니다. 반면 RAG는 사용자 입력을 활용하여 먼저 새 데이터 소스에서 정보를 가져오는 정보 검색 구성 요소를 도입합니다. 사용자 쿼리와 관련 정보가 모두 LLM에 제공되어, LLM이 새로운 지식과 학습 데이터를 사용해 더 나은 응답을 생성할 수 있게 합니다. RAG의 주요 이점으로는 최신 정보 제공, 사용자 신뢰 강화, 개발자 제어 강화 등이 있습니다. RAG를 통해 LLM을 라이브 소셜 미디어 피드, 뉴스 사이트 또는 기타 자주 업데이트되는 정보 소스에 직접 연결할 수 있어 사용자에게 최신 정보를 제공할 수 있습니다. 또한 출력에 소스에 대한 인용이나 참조를 포함하여 정확한 정보를 제공함으로써 생성형 AI 솔루션에 대한 신뢰를 높일 수 있습니다.

📚 참조 문서: 3개
------------------------------------------------------------

🔄 테스트 2:
👤 사용자: 그럼 기존 AI 모델과 어떤 차이가 있나요?
🤖 AI: RAG(검색 증강 생성)는 기존 AI 모델, 특히 대규모 언어 모델(LLM)과 다음과 같은 주요 차이점이 있습니다:
